# Multicriteria Spatial Analysis: Ideal retirement areas in Colombia  

Geoprocesamiento 2025-1S  
Universidad Nacional de Colombia  
Authors: Alicia Arévalo Robinson and Ricardo Andrés Martinez Soto  
Link to repository:

#### **Objective**
To identify the most suitable areas for retirement living in Colombia, considering climate, accessibility, medical services, and the quality of the natural environment.